## Importing libraries

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
import emoji

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading datasets

In [4]:
df_1 = pd.read_csv('data/db_AGP_2019_sucesos.csv')
df_2 = pd.read_csv('data/postID_suceso.csv')

## Exploring dataset

In [5]:
df_1.head()

,created_time.1,created_time.2,message,id,object_link.connections.comments.message,FECHA Y HORA
0,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🧐🧐,19/04/2019 9:16
1,"jueves, 18 de abril de 2019",30/12/1899 20:57:16,"Exequias del ex presidente Alan García en ""La ...",71263708835_276236269920595,🥺🥺🥺🥺🥺🥺🥺🥺,18/04/2019 20:57
2,"miércoles, 17 de abril de 2019",30/12/1899 07:37:30,🚨 Estamos en los exteriores del Hospital de Em...,71263708835_310239669653341,🥳,17/04/2019 7:37
3,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤷‍♀️🤷‍♀️🤷‍♀️,19/04/2019 9:16
4,"viernes, 19 de abril de 2019",30/12/1899 09:16:15,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤯🤯🤯🤯,19/04/2019 9:16


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55006 entries, 0 to 55005
Data columns (total 6 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   created_time.1                            55006 non-null  object
 1   created_time.2                            55006 non-null  object
 2   message                                   55006 non-null  object
 3   id                                        55006 non-null  object
 4   object_link.connections.comments.message  55006 non-null  object
 5   FECHA Y HORA                              55006 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [7]:
df_2.head()

,POST NRO,SUCESO
0,71263708835_310239669653341,Traslado a la Clinica
1,71263708835_10157689377373836,Confirmación de Intento de Suicidio
2,71263708835_2430897753596693,Confirmación de Intento de Suicidio
3,71263708835_10157689601593836,Confirmación de Intento de Suicidio
4,71263708835_426911824739777,Confirmación de Intento de Suicidio


In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   POST NRO  87 non-null     object
 1   SUCESO    87 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


## Cleaning data

In [9]:
# drop cols
df_1.drop(columns=['created_time.2','created_time.1'],inplace=True)

# merge the both datasets
df = df_1.merge(df_2,how='left',left_on = 'id',right_on='POST NRO')

# drop redundant cols
df.drop(columns=['id','POST NRO'],inplace=True)

In [10]:
# change the names of the columns
df.columns = ['post','comment','datetime','context']

# order the columns
df = df[['datetime','context','post','comment']]

In [11]:
# change the type of 'datetime'
df['datetime'] = pd.to_datetime(df['datetime'],format='%d/%m/%Y %H:%M')

In [12]:
# check nan values
df.isnull().sum()

datetime    0
context     0
post        0
comment     0
dtype: int64

In [13]:
# check duplicated values
df.duplicated().sum()

np.int64(1444)

In [14]:
# drop duplicated values
df.drop_duplicates(inplace=True)

In [15]:
# Reset index
df.reset_index(drop=True, inplace=True)

In [16]:
punct = string.punctuation
df['comment'] = df['comment'].str.replace(f'[{punct}]','',regex=True).str.lower()

In [17]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['comment'] = df['comment'].apply(remove_stopwords)

In [18]:
import unicodedata

def remove_accents(text):
    text = unicodedata.normalize('NFKD', text)
    text_without_accents = ''.join(c for c in text if not unicodedata.combining(c))
    return text_without_accents

df['comment'] = df['comment'].apply(remove_accents)

In [19]:
emoji_to_spanish = {
    "©": "copyright ",
    "®": "registrado ",
    "‼": "sorpresa ",
    "⁉": "duda ",
    "▪": "cuadro negro ",
    "☀": "sol ",
    "☕": "cafe ",
    "☝": "arriba ",
    "☠": "peligro ",
    "☹": "triste ",
    "☺": "feliz ",
    "♀": "mujer ",
    "♂": "hombre ",
    "♥": "amor ",
    "⚖": "justicia ",
    "⚠": "advertencia ",
    "⚰": "ataud ",
    "⚽": "futbol ",
    "⛏": "pico ",
    "⛪": "iglesia ",
    "✅": "verificado ",
    "✈": "avión ",
    "✊": "puño ",
    "✋": "mano ",
    "✌": "paz ",
    "✍": "escribir ",
    "✔": "correcto ",
    "✝": "cruz ",
    "✨": "brillo ",
    "❌": "incorrecto ",
    "❎": "cancelado ",
    "❗": "exclamacion ",
    "❤": "corazon ",
    "➕": "mas ",
    "⬆": "arriba ",
    "⬇": "abajo ",
    "⭐": "estrella ",
    "〰": "línea_ondulada ",
    "🅱": "b ",
    "🆘": "emergencia ",
    "🌊": "ola ",
    "🌌": "galaxia ",
    "🌍": "tierra ",
    "🌏": "planeta ",
    "🌞": "sol ",
    "🌟": "estrella ",
    "🌠": "estrella fugaz ",
    "🌱": "planta ",
    "🌶": "picante ",
    "🌷": "flor ",
    "🌹": "rosa ",
    "🌻": "girasol ",
    "🌿": "hierba ",
    "🍃": "hoja ",
    "🍒": "cereza ",
    "🍯": "miel ",
    "🍺": "cerveza ",
    "🍻": "brindis ",
    "🍾": "champan ",
    "🍿": "palomitas ",
    "🎆": "fuegos artificiales ",
    "🎇": "chispa ",
    "🎉": "celebracion ",
    "🎊": "fiesta ",
    "🎓": "graduacion ",
    "🎛": "controles ",
    "🎥": "cine ",
    "🎪": "circo ",
    "🎬": "claqueta ",
    "🎭": "teatro ",
    "🎵": "nota ",
    "🎶": "musica ",
    "🎼": "partitura ",
    "🏃": "correr ",
    "🏆": "trofeo ",
    "🏌": "golf ",
    "🏙": "ciudad ",
    "🏝": "isla ",
    "🏡": "casa ",
    "🏥": "hospital ",
    "🏻": "piel clara ",
    "🏼": "piel media clara ",
    "🏽": "piel media ",
    "🏾": "piel media oscura ",
    "🏿": "piel oscura ",
    "🐀": "rata ",
    "🐁": "raton ",
    "🐈": "gato ",
    "🐎": "caballo ",
    "🐑": "oveja ",
    "🐖": "cerdo ",
    "🐗": "jabali ",
    "🐤": "pollito ",
    "🐥": "pollito feliz ",
    "🐭": "raton",
    "🐱": "gato",
    "🐴": "caballo ",
    "🐶": "perro ",
    "🐷": "cerdo ",
    "🐺": "lobo ",
    "👀": "ojos ",
    "👁": "ojo ",
    "👂": "oido ",
    "👅": "lengua ",
    "👆": "arriba ",
    "👇": "abajo ",
    "👈": "izquierda ",
    "👉": "derecha ",
    "👊": "puño ",
    "👋": "saludo ",
    "👌": "ok ",
    "👍": "pulgar arriba ",
    "👎": "pulgar abajo ",
    "👏": "aplauso ",
    "👐": "manos abiertas ",
    "👚": "blusa ",
    "👝": "bolso ",
    "👨": "hombre ",
    "👪": "familia ",
    "👹": "ogro ",
    "👺": "demonio ",
    "👻": "fantasma ",
    "👼": "angel ",
    "👽": "alien ",
    "👾": "extraterrestre ",
    "👿": "diablo ",
    "💀": "calavera ",
    "💁": "persona indicando ",
    "💃": "bailando ",
    "💆": "masaje ",
    "💐": "flores ",
    "💒": "boda ",
    "💓": "corazon",
    "💔": "corazon roto ",
    "💕": "corazones ",
    "💖": "corazon ",
    "💗": "corazon ",
    "💘": "corazon ",
    "💚": "corazon ",
    "💜": "corazon ",
    "💡": "idea ",
    "💣": "bomba ",
    "💤": "dormir ",
    "💥": "explosion ",
    "💦": "gotas ",
    "💨": "viento ",
    "💩": "popo ",
    "💪": "fuerza ",
    "💫": "mareo ",
    "💭": "pensamiento ",
    "💯": "perfecto ",
    "💰": "dinero ",
    "💲": "dolar ",
    "💳": "tarjeta ",
    "💵": "billete ",
    "💶": "euro ",
    "💷": "libra ",
    "💹": "crecimiento ",
    "📢": "megafono ",
    "📣": "altavoz ",
    "📸": "foto ",
    "📺": "television ",
    "🔊": "volumen alto ",
    "🔐": "cerrado ",
    "🔥": "fuego ",
    "🔨": "martillo ",
    "🔪": "cuchillo ",
    "🔫": "pistola ",
    "🔱": "tridente ",
    "🔴": "circulo rojo ",
    "🔵": "circulo azul ",
    "🔷": "rombo azul ",
    "🕊": "paloma ",
    "🕯": "vela ",
    "🕵": "detective ",
    "🖐": "mano ",
    "🖕": "dedo_medio ",
    "🖤": "corazon negro ",
    "🗑": "basura ",
    "🗼": "torre ",
    "😀": "sonrisa ",
    "😁": "carcajada ",
    "😂": "risa_llorando ",
    "😃": "alegria ",
    "😄": "risa ",
    "😅": "alivio ",
    "😆": "risa ",
    "😇": "angel ",
    "😈": "diablo ",
    "😉": "guiño ",
    "😊": "contento ",
    "😋": "sabroso ",
    "😌": "tranquilo ",
    "😍": "enamorado ",
    "😎": "cool ",
    "😏": "satisfecho ",
    "😐": "neutral ",
    "😑": "indiferente ",
    "😒": "decepcionado ",
    "😓": "sudor ",
    "😔": "pensativo ",
    "😕": "confuso ",
    "😖": "preocupado ",
    "😗": "beso ",
    "😘": "beso ",
    "😙": "beso ",
    "😚": "beso ",
    "😛": "lengua fuera ",
    "😜": "lengua guiño ",
    "😝": "lengua broma ",
    "😞": "frustrado ",
    "😟": "angustiado ",
    "😠": "enojado ",
    "😡": "furioso ",
    "😢": "llorar ",
    "😣": "perseverante ",
    "😤": "determinacin ",
    "😥": "aliviado ",
    "😦": "preocupacion ",
    "😧": "shock ",
    "😨": "temeroso ",
    "😩": "agotado ",
    "😪": "somnoliento ",
    "😫": "agotamiento ",
    "😬": "incomodo ",
    "😭": "llorar ",
    "😮": "sorpresa ",
    "😯": "silencio ",
    "😰": "ansiedad ",
    "😱": "grito ",
    "😲": "impactado ",
    "😳": "avergonzado ",
    "😴": "dormido ",
    "😵": "mareado ",
    "😶": "sin palabras ",
    "😷": "enfermo ",
    "😸": "gato feliz ",
    "😹": "gato risa ",
    "😼": "gato malicioso ",
    "😿": "gato triste ",
    "🙁": "decepcionado ",
    "🙂": "feliz ",
    "🙃": "al reves ",
    "🙄": "ojo volteado ",
    "🙅": "prohibido ",
    "🙆": "ok ",
    "🙇": "reverencia ",
    "🙈": "no ver ",
    "🙉": "no escuchar ",
    "🙊": "no hablar ",
    "🙋": "mano alzada ",
    "🙌": "victoria ",
    "🙏": "oracion ",
    "🚆": "tren ",
    "🚑": "ambulancia ",
    "🚫": "prohibido ",
    "🚶": "caminar ",
    "🛥": "barco ",
    "🛩": "avion ",
    "🛫": "despegue ",
    "🛬": "aterrizaje ",
    "🛸": "ovni ",
    "🤐": "silencio ",
    "🤑": "dinero ",
    "🤒": "enfermo ",
    "🤓": "nerd ",
    "🤔": "pensativo ",
    "🤕": "herido ",
    "🤗": "abrazo ",
    "🤘": "rock ",
    "🤙": "shaka ",
    "🤞": "cruzar dedos ",
    "🤟": "te_amo ",
    "🤡": "payaso ",
    "🤢": "asco ",
    "🤣": "carcajada ",
    "🤤": "baba ",
    "🤥": "mentira ",
    "🤦": "frustracion ",
    "🤧": "estornudo ",
    "🤨": "sospecha ",
    "🤩": "estrellas ",
    "🤪": "loco ",
    "🤫": "secreto ",
    "🤬": "molesto ",
    "🤭": "sorpresa ",
    "🤮": "vomito ",
    "🤯": "impacto ",
    "🤶": "navidad ",
    "🤷": "duda ",
    "🤹": "malabarista ",
    "🥀": "flor muerta ",
    "🥂": "brindis ",
    "🥃": "trago ",
    "🥑": "aguacate ",
    "🥚": "huevo ",
    "🥳": "celebracion ",
    "🥴": "mareado ",
    "🥵": "calor ",
    "🥶": "frio ",
    "🥺": "suplicar ",
    "🦁": "leon ",
    "🦃": "pavo ",
    "🦆": "pato ",
    "🦊": "zorro ",
    "🦸": "heroe ",
    "🧐": "curioso ",
    "🧠": "cerebro ",
    "🧬": "adn ",
    "🧹": "escoba "
}

In [20]:
def separate_text_from_emojis(text):
    list_text_without_emojis = []
    list_text_with_emojis = []
    
    for char in text:
        if char not in emoji.EMOJI_DATA:
            list_text_without_emojis.append(char)
        else:
            list_text_with_emojis.append(char)
    return "".join(list_text_with_emojis), "".join(list_text_without_emojis)

In [21]:
new_comment = df['comment'].apply(separate_text_from_emojis)

emojis_in_comment, text_in_comment = [], []

for a,b in new_comment:
    emojis_in_comment.append(a)
    text_in_comment.append(b)

In [22]:
def translate_emojis_to_spanish(text):
    if not isinstance(text, str):
        return ""
    
    list_text = []
    for char in text:
        if char in emoji.EMOJI_DATA:
            list_text.append(emoji_to_spanish.get(char, "desconocido "))
        else:
            list_text.append(char)
    
    return "".join(list_text)

In [23]:
emojis_text = []

for text in emojis_in_comment:
    emojis_text.append(translate_emojis_to_spanish(text))

In [24]:
text = []
for text_1, text_2 in zip(text_in_comment, emojis_text):
    text.append(text_1+ ' '+ text_2)

In [25]:
df['new_comment'] = pd.Series(text)
df['new_comment'] = df['new_comment'].apply(remove_accents)

## Sentimental Analysis

In [27]:
import pandas as pd
lexicon = pd.read_csv('data/espaniol_NRC.csv', index_col='Spanish Word')
lexicon.drop(columns=['anticipacion','positivo','confianza','negativo'], inplace=True)

def feelings_in_text(text):
    feelings_count = {col: 0 for col in lexicon.columns}
    words = text.split()

    for word in words:
        if word in lexicon.index:
            feeling_values = lexicon.loc[word].values
            for i, value in enumerate(feeling_values):
                feelings_count[lexicon.columns[i]] += value

    feelings, count = sorted(feelings_count.items(), key=lambda item: item[1], reverse=True)[0]
    if count == 0:
        return 'neutral'
    else: 
        return feelings

In [28]:
temp_data = df['new_comment'].apply(feelings_in_text)

In [29]:
df['sentiments'] = temp_data

In [30]:
df['day'] = df['datetime'].apply(lambda x: x.day)

In [31]:
df.drop(columns=['post','comment'],inplace=True)

In [41]:
df

,datetime,context,new_comment,sentiments,day
0,2019-04-19 09:16:00,Lectura de Carta de AG,curioso curioso,neutral,19
1,2019-04-18 20:57:00,Velatorio Casa del Pueblo,suplicar suplicar suplicar suplicar suplicar ...,neutral,18
2,2019-04-17 07:37:00,Traslado a la Clinica,celebracion,alegria,17
3,2019-04-19 09:16:00,Lectura de Carta de AG,‍️‍️‍️ duda mujer duda mujer duda mujer,miedo,19
4,2019-04-19 09:16:00,Lectura de Carta de AG,impacto impacto impacto impacto,neutral,19
...,...,...,...,...,...
53557,2019-04-18 22:01:00,Portadas del día,comercio asi saques primera pagina dr alan gar...,disgusto,18
53558,2019-04-19 12:35:00,Ataud Sale del local APRA,xq nadie puede ver cadaver,disgusto,19
53559,2019-04-18 19:12:00,Velatorio Casa del Pueblo,– 1985 69 personas asesinadas accomarca – 1986...,miedo,18
53560,2019-04-18 19:12:00,Velatorio Casa del Pueblo,– 1985 69 personas asesinadas accomarca – 1986...,miedo,18


In [ ]:
df.to_csv('clean_data.csv',index=False)